# Quickstart

A tour of this package's functionality! Buckle up because it's gonna be a fun one.

## Tools and Toolkits

### Tools: `@function_tool`
You can make any function into a tool usable by the model by decorating it with `@function_tool`, but you must do the following:
- The parameters must be type annotated
- The return type *should* be str, but not required (currently just stringified)
- It must be documented with a `'''docstring'''`, including each parameter (most [formats supported](https://github.com/rr-/docstring_parser), e.g. [Google-style](https://gist.github.com/redlotus/3bc387c2591e3e908c9b63b97b11d24e#file-docstrings-py-L67), [NumPy-style](https://gist.github.com/eikonomega/910512d92769b0cc382a09ae4de41771), sphinx-style, etc, see [this overview](https://gist.github.com/nipunsadvilkar/fec9d2a40f9c83ea7fd97be59261c400))

You can use the tool from python as you normally would, but also the annotated tool will be auto-parsed and its JSON schema and argument validator (using pydantic) will be available as attributes on the function:
- `tool.schema` is the (singular list of) JSON schema, which you can directly pass to the OpenAI API.
- `tool.lookup` is a dict lookup table of the function name to the (auto-validated) function call, meaning you can just receive the model-generated function call and pass it directly to the tool.

Other attributes (probably commonly used):
- `tool.validator` is the pydantic validator, callable with kwargs to validate the arguments. This is what is saved in the lookup table.
- `tool.tool_enabled` is a bool switch to enable/disable the tool. This is useful esp. for Toolkits, where you can dynamically enable/disable tools based on the context. Note that this only works if you wrap the tool in `ToolList` class or is part of a `Toolkit`.
- `tool.name` should ideally not be set directly, but passed as a kwarg to `@function_tool`. It is the name of the tool, as known to the model. If not set, it will be the name of the function.

In [1]:
from gpt_wrapper.tools import ToolList, Toolkit, function_tool, fail_with_message
import logging
logger = logging.getLogger(__name__)

In [2]:
@function_tool
def print_to_console(text: str) -> str:
    '''
    Print text to console

    Args:
        text: text to print
    '''
    print(text)
    return 'success' # ideally, we always return something to tell the model

# normal call
print_to_console('Hello from python!')

# call from lookup table
lookup = print_to_console.lookup
# this would be generated by GPT
name, arguments = 'print_to_console', {'text': 'Hello from GPT!'}
func = lookup[name]
func(arguments) # notice the lack of unpacking, we pass the args dict directly

Hello from python!
Hello from GPT!


'success'

In [3]:
from types import SimpleNamespace
import json
from gpt_wrapper.tools import call_requested_function

call = SimpleNamespace(name='print_to_console', arguments=json.dumps({'text': 'Hello from GPT!'}))
await call_requested_function(call, print_to_console.lookup)

awaiting thread
Hello from GPT!


'success'

In [4]:
# if the model generates an invalid argument, it is auto-validated
print(func({'text': 123}))
print(func({'content': "Whats up?"}))

# notice that it's returned as string, because it should be passed to the model to correct itself

Invalid Argument: 1 validation error for print_to_console
text
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
Invalid Argument: 1 validation error for print_to_console
text
  Field required [type=missing, input_value={'content': 'Whats up?'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


Showing off some more goodies:
- Even async functions should seamlessly work, just don't forget to `await` them.
- `@fail_with_message(err)` is a decorator that will catch any exceptions thrown by the function and instead return the error message. This is useful for when you want to handle errors in a more graceful way than just crashing the model. It also takes an optional logger, which by default takes the `print` function, but any callable that takes a string will work, such as `logger.error` from the `logging` module.
- Usually, the `@function_tool` decorator will throw an assertion error if you forget to provide the description for any of the function or their parameters. If you really don't want to provide descriptions for some (or all), maybe because it's so self-explanatory or you need to save tokens, then you can explicitly turn off the docstring parsing by passing `@function_tool(check_description=False)`. This is not recommended, but it's there if you need it.

In [5]:
import asyncio

@function_tool(name="Fibonacci")
@fail_with_message("Error", logger=logger.error)
async def fib(n: int):
    '''
    Calculate the nth Fibonacci number

    Args:
        n: The index of the Fibonacci number to calculate
    '''
    if n < 0:
         raise ValueError("n must be >= 0")
    if n < 2:
        return n
    await asyncio.sleep(0.1)
    # return await fib(n-1) + await fib(n-2)
    # parallel
    return sum(await asyncio.gather(fib(n-1), fib(n-2)))

print(await fib(10), flush=True)
print(await fib.lookup['Fibonacci']({'n': -10}))
fib.schema

55


Tool call fib(n=-10) failed: n must be >= 0


Error: n must be >= 0


[{'type': 'function',
  'function': {'name': 'Fibonacci',
   'description': 'Calculate the nth Fibonacci number',
   'parameters': {'type': 'object',
    'properties': {'n': {'description': 'The index of the Fibonacci number to calculate',
      'type': 'integer'}},
    'required': ['n']}}}]

### Toolkits: `class Toolkit`
Toolkits are a collection of related function tools, esp. useful when they share a state. Also good for keeping the state bound to a single instance of the toolkit, rather than a global state.
To create a toolkit, simply subclass `Toolkit` and decorate its methods with `@function_tool`.
Enabled tools will be visible in the attributes:
- `toolkit.schema` is the JSON schema, which you can directly pass to the OpenAI API.
- `toolkit.lookup` is a dict lookup table of the function name to the (auto-validated) function call, meaning you can just receive the model-generated function call and pass it directly to the tool.

As you noticed, the interface for OpenAI API is the same for both tools and toolkits, so you can use them interchangeably, and to use multiple tools and toolkits, simply merge their schemas and lookups.

In [6]:
class Notepad(Toolkit):
    def __init__(self):
        super().__init__()
        self.content = "<Fill me in>"
    
    @function_tool
    def write(self, text: str):
        '''
        Write text to the notepad

        Args:
            text: The text to write
        '''
        self.content = text
    
    @function_tool(check_description=False)
    def read(self):
        return self.content
    
notes = Notepad()
notes.write("Hello, world!")
print(notes.read())

Hello, world!


In [7]:
notes.lookup['write']({'text': "Shhh... here's a secret: 42"})
notes.lookup['read']({})

"Shhh... here's a secret: 42"

## ChatGPT model and MessageHistory

### ChatGPT: `class ChatGPT`

Simple wrapper that keeps a message history object and optional toolkit (TODO: make into list of toolkits).
The object is callable and simply takes a user prompt string and returns the model response string.

### MessageHistory: `class MessageHistory`

A simple class that keeps track of the message history, including the user and assistnat messages, along with system and tool call results. Special subclasses could be defined for things like:
- Rolling history (e.g. last 10 messages)
- Dynamic history based on context (e.g. vector embedding)
- System message that constantly updates (e.g. time, weather, etc.)
- Few-shot learning, i.e. pre-populating the history with some demonstration messages

In [8]:
from gpt_wrapper.models import ChatGPT
from gpt_wrapper.messages import MessageHistory, msg

gpt = ChatGPT(
    model = 'gpt-3.5-turbo',
    tools = ToolList(notes, print_to_console, fib),
    messages = MessageHistory()
)

In [12]:
response = await gpt("What's on my notepad?")
print(response)

[Chat Start]: What's on my notepad?
Closing OpenAI stream...
Unhandled Event: Assistant.Completion(completion=ChatCompletion(id='chatcmpl-8VVbUoNqDSdKOU4ipqmXxa3BGKFBW', choices=[Choice(finish_reason='tool_calls', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fx0ZBZ5Gdxmf6UWFTLcP66HS', function=Function(arguments='{}', name='read'), type='function')]))], created=1702520344, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=None), call_index=0)
Unhandled Event: Assistant.ChatMessage(message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fx0ZBZ5Gdxmf6UWFTLcP66HS', function=Function(arguments='{}', name='read'), type='function')]), choice_index=0)
[Tool Calls]: [ChatCompletionMessageToolCall(id='call_Fx0ZBZ5Gdxmf6UWFTLcP66HS', function=Function(arguments='{}', name='read'), type='fun

In [13]:
response = await gpt("Can you calculate the 8th fibonacci number, add it to the number in my notes, and write it? also print it to console as well.")
# print(response)

[Chat Start]: Can you calculate the 8th fibonacci number, add it to the number in my notes, and write it? also print it to console as well.
Closing OpenAI stream...
Unhandled Event: Assistant.Completion(completion=ChatCompletion(id='chatcmpl-8VVcsygkDQcVChuAM0KG7rKMYFoTb', choices=[Choice(finish_reason='tool_calls', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8B3KC2PclAkyDiyZpPNlQU30', function=Function(arguments='{\n  "n": 8\n}', name='Fibonacci'), type='function')]))], created=1702520430, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=None), call_index=0)
Unhandled Event: Assistant.ChatMessage(message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8B3KC2PclAkyDiyZpPNlQU30', function=Function(arguments='{\n  "n": 8\n}', name='Fibonacci'), type='function')]), choice_index=0)


In [14]:
notes.read()

'63'

In [15]:
gpt.messages.history

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': "What's on my notepad?"},
 {'role': 'user', 'content': "What's on my notepad?"},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_Fx0ZBZ5Gdxmf6UWFTLcP66HS',
    'function': {'arguments': '{}', 'name': 'read'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': "Shhh... here's a secret: 42",
  'tool_call_id': 'call_Fx0ZBZ5Gdxmf6UWFTLcP66HS'},
 {'content': 'On your notepad, there is a secret message that says: "42"',
  'role': 'assistant'},
 {'role': 'user',
  'content': 'Can you calculate the 8th fibonacci number, add it to the number in my notes, and write it? also print it to console as well.'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_8B3KC2PclAkyDiyZpPNlQU30',
    'function': {'arguments': '{\n  "n": 8\n}', 'name': 'Fibonacci'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '21',
  'tool_call_id': 'call_8B3KC2PclAkyDiyZpPNlQU30'},
 {'role': 'assistant',
  't

### Customizing our ChatGPT Class

If we want to hook up our class to a front-end for example, we'd want to allow streaming. We can easily override one of the many high-level functions and user hooks. In this case, we can override the `request_chat` function which is supposed to return the complete response. Before returning the response, we can take the streaming response and print it before returning the final response.

In [25]:
from gpt_wrapper.api import openai_chat, accumulate_partial

class StreamingChatGPT(ChatGPT):
    async def request_chat(self, messages, tools, model, **openai_kwargs):
        openai_kwargs['stream'] = True
        completion_stream = await openai_chat(messages=messages, tools=tools.schema, model=model, **openai_kwargs)

        async for partial in accumulate_partial(completion_stream):
            try:
                message = await self.select_message(partial)

                if message.content:


            except Exception:
                print('ignoring exception')

        return partial

gpt = StreamingChatGPT(
    model = 'gpt-3.5-turbo',
    tools = ToolList(notes, print_to_console, fib),
    messages = MessageHistory()
)

In [26]:
await gpt("Give me a recursive python function that calculates the nth fibonacci number")

cChatCompletion(id='chatcmpl-8St7Qh7wHS4c3PW0XafTcR6eC3nKp', choices=[Choice(finish_reason='length', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=None))], created=1701895752, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=None)
cChatCompletion(id='chatcmpl-8St7Qh7wHS4c3PW0XafTcR6eC3nKp', choices=[Choice(finish_reason='length', index=0, message=ChatCompletionMessage(content='Certainly', role='assistant', function_call=None, tool_calls=None))], created=1701895752, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=None)
cChatCompletion(id='chatcmpl-8St7Qh7wHS4c3PW0XafTcR6eC3nKp', choices=[Choice(finish_reason='length', index=0, message=ChatCompletionMessage(content='Certainly!', role='assistant', function_call=None, tool_calls=None))], created=1701895752, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=None)
cChatComplet

'Certainly! Here\'s a recursive Python function that calculates the nth Fibonacci number:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return "Invalid input"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n```\n\nYou can use this function to calculate the nth Fibonacci number by passing the desired index as an argument to the `fibonacci` function. For example, `fibonacci(5)` will return the 5th Fibonacci number.'